In [69]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import os
import sqlite3

Para saber cuales han sido tanto los pilotos ganadores de los campeonatos como los equipos ganadores he usado una tabla de la siguiente página web. La tabla contiene un total de 75 filas, desde 1950 hasta 2024 son todos los campeonatos de F1 que se han disputado. La tabla contiene datos como el nombre del piloto ganador, el equipo con el que corrió, cuantas poles consiguió, vueltas rápidas y en qué carrera consiguió el campeonato matemáticamente.

In [70]:
url = "https://en.wikipedia.org/wiki/List_of_Formula_One_World_Drivers%27_Champions"
# Obtener el contenido de la página
response = requests.get(url)

# Verificar si la solicitud fue exitosa
print(response.status_code)

200


In [71]:
# Extraer tabla de datos de los ganadores de cada año.
url = "https://en.wikipedia.org/wiki/List_of_Formula_One_World_Drivers%27_Champions"

# Obtener el contenido de la página
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Buscar la tabla con la clase correcta
table = soup.find("table", {"class": "wikitable sortable"})

# Extraer las filas de la tabla
filas = table.find_all("tr")

#Lista para almacenar los datos
data = []

for fila in filas[1:]:  # Ignorar la primera fila que es el encabezado
    cols = fila.find_all("td")
    
    if len(cols) == 16:
        year = cols[0].text.strip()
        driver = cols[1].text.strip()
        age = cols[2].text.strip()
        team = cols[3].text.strip()
        engine = cols[4].text.strip()
        tyres = cols[5].text.strip()
        pole = cols[6].text.strip()
        wins = cols[7].text.strip()
        podiums = cols[8].text.strip()
        fastest_laps = cols[9].text.strip()
        points = cols[10].text.strip()
        percent_points = cols[11].text.strip()
        clinched = cols[12].text.strip()
        rounds_remaining = cols[13].text.strip()
        margin = cols[14].text.strip()
        percent_margin = cols[15].text.strip()
        
        # Agregar los datos a la lista
        data.append([year, driver, age, team, engine, tyres, pole, wins, 
                     podiums, fastest_laps, points, percent_points, clinched, 
                     rounds_remaining, margin, percent_margin])

df_temporadas_f1 = pd.DataFrame(data, columns=[
    "year", "Driver", "Age", "Team", "Engine", "Tyres", "Pole", 
    "Wins", "Podiums", "Fastest Laps", "Points", "% Points", "Clinched", 
    "# of Rounds Remaining", "Margin", "% Margin"
])

# Eliminar los corchetes y contenido de las columnas que tienen corchetes para limpiar los datos.
for column in df_temporadas_f1.columns:
    for index in range(len(df_temporadas_f1)):
        df_temporadas_f1.at[index, column] = re.sub(r'\[.*?\]', '', str(df_temporadas_f1.at[index, column]))

df_temporadas_f1["year"] = pd.to_numeric(df_temporadas_f1["year"])
df_temporadas_f1

,year,Driver,Age,Team,Engine,Tyres,Pole,Wins,Podiums,Fastest Laps,Points,% Points,Clinched,# of Rounds Remaining,Margin,% Margin
0,1950,Giuseppe Farina,44,Alfa Romeo,Alfa Romeo,P,2,3,3,3,30,83.333 (47.619),Round 7 of 7,0,3,10.000
1,1951,Juan Manuel Fangio,40,Alfa Romeo,Alfa Romeo,P,4,3,5,5,31,86.111 (51.389),Round 8 of 8,0,6,19.355
2,1952,Alberto Ascari,34,Ferrari,Ferrari,F P,5,6,6,6,36,100.000 (74.306),Round 6 of 8,2,12,33.333
3,1953,Alberto Ascari,35,Ferrari,Ferrari,P,6,5,5,4,34.5,95.833 (57.407),Round 8 of 9,1,6.5,18.841
4,1954,Juan Manuel Fangio,43,Maserati,Maserati,P,5,6,7,3,42,93.333 (70.547),Round 7 of 9,2,16.857,40.136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,2020,Lewis Hamilton,35,Mercedes,Mercedes,P,10,11,14,6,347,78.507,Round 14 of 17,3,124,35.735
71,2021,Max Verstappen,24,Red Bull,Honda,P,10,10,18,6,395.5,69.692,Round 22 of 22,0,8,2.023
72,2022,Max Verstappen,25,Red Bull,RBPT,P,7,15,17,5,454,76.174,Round 18 of 22,4,146,32.159
73,2023,Max Verstappen,26,Red Bull,Honda RBPT,P,12,19,21,9,575,92.742,Round 17 of 22,5,290,50.435


In [72]:
circuits = pd.read_csv("../data/f1_data/circuits.csv")
constructors = pd.read_csv("../data/f1_data/constructors.csv")
constructor_results = pd.read_csv("../data/f1_data/constructor_results.csv")
constructor_standings = pd.read_csv("../data/f1_data/constructor_standings.csv")
drivers = pd.read_csv("../data/f1_data/drivers.csv")
driver_standings = pd.read_csv("../data/f1_data/driver_standings.csv")
lap_times = pd.read_csv("../data/f1_data/lap_times.csv")
pit_stops = pd.read_csv("../data/f1_data/pit_stops.csv")
qualifying = pd.read_csv("../data/f1_data/qualifying.csv")
races = pd.read_csv("../data/f1_data/races.csv")
results = pd.read_csv("../data/f1_data/results.csv")
seasons = pd.read_csv("../data/f1_data/seasons.csv")
status = pd.read_csv("../data/f1_data/status.csv")

# Paso 1: Unir resultados con las carreras
df_combinado = pd.merge(results, races, on='raceId', how='left')

# Paso 2: Agregar información de los pilotos
df_combinado = pd.merge(df_combinado, drivers, on='driverId', how='left')

# Paso 3: Agregar información de los constructores si la columna 'constructorId' existe
if 'constructorId' in constructors.columns:
    df_combinado = pd.merge(df_combinado, constructors, on='constructorId', how='left')

# Paso 4: Agregar resultados de los constructores si la columna 'constructorId' existe
if 'constructorId' in constructor_results.columns:
    df_combinado = pd.merge(df_combinado, constructor_results, on=['raceId', 'constructorId'], how='left')

# Paso 5: Agregar standings de los constructores si la columna 'constructorId' existe
if 'constructorId' in constructor_standings.columns:
    df_combinado = pd.merge(df_combinado, constructor_standings, on=['raceId', 'constructorId'], how='left')


# Paso 7: Agregar clasificación si las columnas 'raceId' y 'driverId' existen
if 'raceId' in qualifying.columns and 'driverId' in qualifying.columns:
    df_combinado = pd.merge(df_combinado, qualifying, on=['raceId', 'driverId'], how='left', suffixes=('', '_qualifying'))

# Al final, elimina columnas duplicadas si es necesario, o conserva solo una versión
df_combinado = df_combinado.loc[:, ~df_combinado.columns.str.endswith('_qualifying')]

# Paso 8: Agregar standings de los pilotos si las columnas 'raceId' y 'driverId' existen
if 'raceId' in driver_standings.columns and 'driverId' in driver_standings.columns:
    df_combinado = pd.merge(df_combinado, driver_standings, on=['raceId', 'driverId'], how='left', suffixes=('', '_driver_standings'))

# Eliminar las columnas duplicadas si ya existen en el DataFrame
df_combinado = df_combinado.loc[:, ~df_combinado.columns.str.endswith('_driver_standings')]

# Paso 9: Agregar temporadas si la columna 'year' existe
if 'year' in seasons.columns:
    df_combinado = pd.merge(df_combinado, seasons, on='year', how='left', suffixes=('', '_seasons'))

    # Eliminar las columnas duplicadas si ya existen en el DataFrame
    df_combinado = df_combinado.loc[:, ~df_combinado.columns.str.endswith('_seasons')]
    
# Paso 10: Agregar estado de los pilotos si la columna 'statusId' existe
if 'statusId' in status.columns:
    df_combinado = pd.merge(df_combinado, status, on='statusId', how='left')

# Eliminar filas duplicadas
df_combinado = df_combinado.drop_duplicates()
pd.set_option("display.max_columns", None)

# Eliminar colunmas que no nos interesan.
df_combinado.drop(columns=["url_x","url_y","url","quali_date","quali_time","sprint_date","sprint_time",
                          "name_y","surname","fp1_date","fp1_time","fp2_date","fp2_time",
                          "fp3_date","fp3_time","milliseconds","time_y","positionText_x","number_y",
                          "positionText_y","driverId","qualifyId","number","statusId","resultId","constructorId"], inplace = True)

# Renombrar columnas para entenderlar.
df_combinado.rename(columns={"nationality_y":"team_nation","nationality_x":"driver_nation","constructorRef":"team",
                            "date":"race_date","name_x":"grand_prix_name","time_x":"driver_race_time",
                            "position_x":"driver_race_position","number_x":"car_number","points_x":"driver_points",
                            "wins":"team_wins","points_y":"team_points","position_y":"team_position",
                            "status_y":"driver_race_status"}, inplace=True)

df_combinado

,raceId,car_number,grid,driver_race_position,positionOrder,driver_points,laps,driver_race_time,fastestLap,rank,fastestLapTime,fastestLapSpeed,year,round,circuitId,grand_prix_name,race_date,driverRef,code,forename,dob,driver_nation,team,team_nation,constructorResultsId,team_points,status_x,constructorStandingsId,points,team_position,team_wins,position,q1,q2,q3,driverStandingsId,positionText,driver_race_status
0,18,22,1,1,1,10.0,58,1:34:50.616,39,2,1:27.452,218.300,2008,1,1,Australian Grand Prix,2008-03-16,hamilton,HAM,Lewis,1985-01-07,British,mclaren,British,1.0,14.0,\N,1.0,14.0,1.0,1.0,1.0,1:26.572,1:25.187,1:26.714,1.0,1,Finished
1,18,3,5,2,2,8.0,58,+5.478,41,3,1:27.739,217.586,2008,1,1,Australian Grand Prix,2008-03-16,heidfeld,HEI,Nick,1977-05-10,German,bmw_sauber,German,2.0,8.0,\N,2.0,8.0,3.0,0.0,5.0,1:25.960,1:25.518,1:27.236,2.0,2,Finished
2,18,7,7,3,3,6.0,58,+8.163,41,5,1:28.090,216.719,2008,1,1,Australian Grand Prix,2008-03-16,rosberg,ROS,Nico,1985-06-27,German,williams,British,3.0,9.0,\N,3.0,9.0,2.0,0.0,7.0,1:26.295,1:26.059,1:28.687,3.0,3,Finished
3,18,5,11,4,4,5.0,58,+17.181,58,7,1:28.603,215.464,2008,1,1,Australian Grand Prix,2008-03-16,alonso,ALO,Fernando,1981-07-29,Spanish,renault,French,4.0,5.0,\N,4.0,5.0,4.0,0.0,12.0,1:26.907,1:26.188,\N,4.0,4,Finished
4,18,23,3,5,5,4.0,58,+18.014,43,1,1:27.418,218.385,2008,1,1,Australian Grand Prix,2008-03-16,kovalainen,KOV,Heikki,1981-10-19,Finnish,mclaren,British,1.0,14.0,\N,1.0,14.0,1.0,1.0,3.0,1:25.664,1:25.452,1:27.079,5.0,5,Finished
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26514,1132,31,18,16,16,0.0,50,\N,46,16,1:30.875,233.371,2024,12,9,British Grand Prix,2024-07-07,ocon,OCO,Esteban,1996-09-17,French,alpine,French,17009.0,0.0,\N,28852.0,9.0,8.0,0.0,18.0,1:34.557,\N,\N,72867.0,18,+2 Laps
26515,1132,11,0,17,17,0.0,50,\N,50,6,1:29.707,236.409,2024,12,9,British Grand Prix,2024-07-07,perez,PER,Sergio,1990-01-26,Mexican,red_bull,Austrian,17001.0,18.0,\N,28843.0,373.0,1.0,7.0,19.0,1:38.348,\N,\N,72852.0,6,+2 Laps
26516,1132,24,14,18,18,0.0,50,\N,43,17,1:31.014,233.014,2024,12,9,British Grand Prix,2024-07-07,zhou,ZHO,Guanyu,1999-05-30,Chinese,sauber,Swiss,17008.0,0.0,\N,28848.0,0.0,10.0,0.0,14.0,1:31.190,1:27.867,\N,72861.0,19,+2 Laps
26517,1132,63,1,\N,19,0.0,33,\N,3,19,1:31.298,232.289,2024,12,9,British Grand Prix,2024-07-07,russell,RUS,George,1998-02-15,British,mercedes,German,17000.0,25.0,\N,28845.0,221.0,4.0,2.0,1.0,1:30.106,1:26.723,1:25.819,72855.0,7,Water pressure


In [73]:
# Creación base de datos con datos de kaggle. Puede que no me sirvan para el análisis.
# Crear una conexión con la base de datos SQLite
conn = sqlite3.connect('f1_database.db')
cursor = conn.cursor()

# Ruta donde se encuentran los archivos CSV
csv_folder_path = "../data/f1_data"  # Cambia esto a la ruta de tus archivos CSV

# Obtener una lista de los archivos CSV en la carpeta
csv_files = [f for f in os.listdir(csv_folder_path) if f.endswith('.csv')]

# Recorrer cada archivo CSV
for file in csv_files:
    # Cargar el archivo CSV en un DataFrame
    df = pd.read_csv(os.path.join(csv_folder_path, file))
    
    # Crear una tabla en la base de datos (si no existe) con el nombre del archivo sin la extensión
    table_name = os.path.splitext(file)[0]
    
    # Convertir los nombres de las columnas a formato compatible con SQL (por ejemplo, sin espacios)
    df.columns = [col.replace(" ", "_") for col in df.columns]
    
    # Crear la tabla con las columnas
    df.to_sql(table_name, conn, if_exists='replace', index=False)

conn.commit()
conn.close()